# ETL (Extract Transform Load)
ETL은 추출(Extract), 변환(Transform), 적재(Load)를 뜻한다. 컴퓨팅에서 데이터베이스 이용의 한 과정으로 특히 데이터 웨어하우스에서 다음을 말한다. 이는 데이터를 다루는 IT쪽 용어이다.
- 동일 기종 또는 타기종의 데이터 소스로부터 데이터를 추출.
- 조회 또는 분석을 목적으로 데이터를 적절한 포맷이나 구조로 데이터를 저장하기 위해 데이터를 변환.
- 변환 데이터를 적재.

# 일단 데이터를 불러서 와서 하나씩 해보자.

In [12]:
import pandas as pd

In [25]:
df = pd.read_csv("ranging_stock_df_python3.csv")

In [26]:
df

,날짜,시가,고가,저가,종가,거래량,종목명,종목코드,시장구분
0,2021-11-01,4200,4245,4100,4110,236625,3S,60310,KOSDAQ
1,2021-11-02,4095,4385,4095,4200,190736,3S,60310,KOSDAQ
2,2021-11-03,4240,4245,4115,4180,204441,3S,60310,KOSDAQ
3,2021-11-04,4205,4215,4110,4115,131216,3S,60310,KOSDAQ
4,2021-11-05,4080,4130,3960,3985,295209,3S,60310,KOSDAQ
...,...,...,...,...,...,...,...,...,...
225,2021-11-25,24200,24250,23700,23900,92853,AP시스템,265520,KOSDAQ
226,2021-11-26,23950,24000,22950,23100,183281,AP시스템,265520,KOSDAQ
227,2021-11-29,22350,22900,22300,22500,153163,AP시스템,265520,KOSDAQ
228,2021-11-30,22750,23100,21000,21300,546017,AP시스템,265520,KOSDAQ


# 1. 추출(Extract)
필요한데이터 칼럼을 추출한다고 이해하도록 하자.<br>
필요한 데이터는 종목명, 종목코드, 시장구분이다 이것을 이용해 나중에 날짜를 넣어 주식데이터를 불러올것이다. 링크는 아래 링크를 참고하자.
- [주식 정보 수집을 위한 사전준비 및 연습](https://uikang.tistory.com/88)
- [[python]외부변수 설정](https://uikang.tistory.com/93)

In [34]:
df_extract = df[["종목명","종목코드","시장구분"]]

In [35]:
df_extract

,종목명,종목코드,시장구분
0,3S,60310,KOSDAQ
1,3S,60310,KOSDAQ
2,3S,60310,KOSDAQ
3,3S,60310,KOSDAQ
4,3S,60310,KOSDAQ
...,...,...,...
225,AP시스템,265520,KOSDAQ
226,AP시스템,265520,KOSDAQ
227,AP시스템,265520,KOSDAQ
228,AP시스템,265520,KOSDAQ


# 2. 변환(Transform)
이제 중복된것을 .drop_duplicates()함수를 이용해 하나만 남겨서 변환하자. <br>
또한 인덱스 순서가 엉망인것을 볼 수 있다. 인덱스 재배열을 하여 변환하자.

In [54]:
df_transform = df_extract[['종목명','종목코드','시장구분']].drop_duplicates()

In [55]:
df_transform

,종목명,종목코드,시장구분
0,3S,60310,KOSDAQ
23,AJ네트웍스,95570,KOSPI
46,AK홀딩스,6840,KOSPI
69,APS홀딩스,54620,KOSDAQ
92,AP시스템,265520,KOSDAQ


In [56]:
df_transform = df_transform.reset_index(drop = True)

In [57]:
df_transform

,종목명,종목코드,시장구분
0,3S,60310,KOSDAQ
1,AJ네트웍스,95570,KOSPI
2,AK홀딩스,6840,KOSPI
3,APS홀딩스,54620,KOSDAQ
4,AP시스템,265520,KOSDAQ


# 3. 적재(Load)
이제 변환이 끝났으니 DB로 적재해보도록 하자. 

In [59]:
import sqlite3 

In [60]:
con = sqlite3.connect("ETL.db")

In [61]:
df_transform.to_sql("ETL_table",con, index=False, if_exists="replace")

In [62]:
cur = con.cursor()

In [63]:
cur.execute("SELECT * FROM ETL_table")

In [64]:
df_load = pd.read_sql("SELECT * FROM ETL_table", con)

In [65]:
df_load

,종목명,종목코드,시장구분
0,3S,60310,KOSDAQ
1,AJ네트웍스,95570,KOSPI
2,AK홀딩스,6840,KOSPI
3,APS홀딩스,54620,KOSDAQ
4,AP시스템,265520,KOSDAQ


In [72]:
con.commit()

In [75]:
con.close()

# 4. 확인

In [77]:
con = sqlite3.connect("ETL.db")

In [79]:
cur = con.cursor()

In [81]:
cur.execute("SELECT * FROM ETL_table")

In [82]:
df_col = pd.read_sql("SELECT * FROM ETL_table", con)

In [83]:
df_col

,종목명,종목코드,시장구분
0,3S,60310,KOSDAQ
1,AJ네트웍스,95570,KOSPI
2,AK홀딩스,6840,KOSPI
3,APS홀딩스,54620,KOSDAQ
4,AP시스템,265520,KOSDAQ


In [84]:
con.close()

이렇게 ETL에 대해서 간단하게 알아보았는데 하지만!!!!!!!! 많은 데이터 용어들이 남았다. 차근차근 알아가며 블로그에 많이 올려두어 기억하도록 하자.